In [1]:
import sys
import os
import django

sys.path.extend(['/Users/a.khalitov/Documents/work/atomhack/back'])
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

if 'setup' in dir(django): 
    django.setup()

In [34]:
from user.models import User, UserPerk
from core.models import Tag, Perk, TagPerk
from tasks.models import Task
from department.models import Department

from django.db.models import Sum, F, Count

In [8]:
import pandas as pd
import random

In [46]:
# Заполняем матриу тегов-перков
perk_list = Perk.objects.all()
tag_list = Tag.objects.all()

table = []
for perk in perk_list:
    for tag in tag_list:
        level = 1
        tag_perk = TagPerk.objects.filter(tag=tag, perk=perk)
        if tag_perk.exists():
            level = tag_perk.first().level
        else:
            TagPerk.objects.create(tag=tag, perk=perk, level=level)
        table.append([perk.name, tag.name, level])

pd.DataFrame(table, columns=['Перк','Тег', 'Уровень'])

,Перк,Тег,Уровень
0,Образование,Учебник,0.95
1,Образование,Парта,0.91
2,Образование,Лопата,0.40
3,ЖКХ,Учебник,0.18
4,ЖКХ,Парта,0.23
5,ЖКХ,Лопата,0.60


In [45]:
# Заполняем матрицу перк-сотрудник
user_list = User.objects.all().exclude(username='admin')
perk_list = Perk.objects.all()

table = []
for perk in perk_list:
    for user in user_list:
        level = 1
        user_perk = UserPerk.objects.filter(user=user, perk=perk)
        if user_perk.exists():
            level = user_perk.first().level
        else:
            UserPerk.objects.create(user=user, perk=perk, level=level)
        table.append([perk.name, user.name, level])
            
pd.DataFrame(table, columns=['Перк','Сотрудник', 'Уровень'])

,Перк,Сотрудник,Уровень
0,Образование,Миша,0.50
1,Образование,Вася,0.70
2,Образование,Петя,0.30
3,ЖКХ,Миша,0.30
4,ЖКХ,Вася,0.05
5,ЖКХ,Петя,0.80


In [75]:
# Считаем, кому дать задачу
def route_task(task):
    perk_tags_level_avg = {}
    for perk in Perk.objects.all().annotate(tags_level_sum=Sum('tagperk__level')):
        perk_tags_level_avg[perk.id] = perk.tags_level_sum
        
    user_list = User.objects\
        .annotate(num_tasks=Count('tasks'))\
        .filter(num_tasks__lte=F('max_slots'))\
        .exclude(username='admin')\
        .order_by('num_tasks')
    
    max_val = 0
    candidates = [] # Юзеры с одинаковой вероятностью
    for user in user_list:
        calculated = 0
        
        for user_perk in UserPerk.objects.filter(user=user):
            calculated += user_perk.level * perk_tags_level_avg[user_perk.perk.id]
            
        print(f'{user.name}: {round(calculated, 2)}')
        if calculated >= max_val:
            max_val = calculated
            if calculated != max_val:
                candidates.clear()
            candidates.append(user)
    
    # Симулируем работу оператора
    if len(candidates) > 0:
        return random.choice(candidates)

In [76]:
task = Task.objects.get(id=1)
print(route_task(task))

 Вася: 1.63
 Миша: 1.43
 Петя: 1.49
 Вася


In [ ]:
def update_user_perks(user, rate):
    user_perks = UserPerk.objects.filter(user=user)
    
    for user_perk in user_perks:
        user_perk.level = user_perk.level + user_perk.level * (rate - ) / (rate + )

In [ ]:
def update_tag_perks(tag, perks):
    pass